In [4]:
import JupyterNotebooksLib as slicernb
import slicer
import numpy as np
import pandas as pd
import os
from tps import ThinPlateSpline

<div class="alert alert-block alert-info">
<b>In Slicer GUI</b> from the MRI-results folder, Load the main volume of the individual and load orientation.mrk.json; from control folder load LinearTransform.h5. Save the project in the control folder.
</div>

## Load data from Control folder

In [5]:
files = [file for file in os.listdir() if '.csv' in file]
bone_dfs = []
for file in files:
    if 'bone' in file:

        df = pd.read_csv(file, index_col='name')
        for body in ['pelvis', 'femur_l', 'femur_r', 'patella_l', 'patella_r', 'tibia_l', 'tibia_r']:
            if body in file:
                df['body'] = body
                bone_dfs.append(df)
bone_df = pd.concat(bone_dfs)
bone_df['group'] = 'bone'

muscle_dfs = []
for file in files:
    if 'muscle' in file:
        df = pd.read_csv(file, index_col='name')
        for body in ['pelvis', 'femur_l', 'femur_r', 'patella_l', 'patella_r', 'tibia_l', 'tibia_r']:
            if body in file:
                df['body'] = body
                muscle_dfs.append(df)
muscle_df= pd.concat(muscle_dfs)
muscle_df['group'] = 'muscle'

wraps_translation_dfs = []
for file in files:
    if 'wraps' in file:
        df = pd.read_csv(file, index_col='name')
        for body in ['pelvis', 'femur_l', 'femur_r', 'patella_l', 'patella_r', 'tibia_l', 'tibia_r']:
            if body in file:
                df['body'] = body
                wraps_translation_dfs.append(df)
wraps_translation_df = pd.concat(wraps_translation_dfs)
wraps_translation_df['group'] = 'wrap_transl'

In [6]:
all_df = pd.concat([bone_df, muscle_df, wraps_translation_df])
all_df.shape

(353, 5)

In [7]:
inds = {'bone':{'all':[]}, 'muscle':{'all':[]}, 'wrap':{'all':[]}}
for i in range(all_df.shape[0]):
    if all_df.iloc[i]['group'] == 'bone':
        inds['bone']['all'].append(i)
        body = all_df.iloc[i]['body']
        if body in inds['bone'].keys():
            inds['bone'][body].append(i)
        else: inds['bone'][body] = [i]
    if all_df.iloc[i]['group'] == 'muscle':
        inds['muscle']['all'].append(i)
        body = all_df.iloc[i]['body']
        if body in inds['muscle'].keys():
            inds['muscle'][body].append(i)
        else: inds['muscle'][body] = [i]
    if all_df.iloc[i]['group'] == 'wrap_transl':
        inds['wrap']['all'].append(i)
        body = all_df.iloc[i]['body']
        if body in inds['wrap'].keys():
            inds['wrap'][body].append(i)
        else: inds['wrap'][body] = [i]


In [9]:
# # load current bone cloud, make pink
# bone_cloud = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode")
# slicer.util.updateMarkupsControlPointsFromArray(bone_cloud, bone_df[['r','a','s']].to_numpy())
# for i, name in enumerate(bone_df.index):
#     bone_cloud.SetNthFiducialLabel(i, name)

In [8]:
# load current muscle cloud, call muscles_orig, make blue
muscle_cloud = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode")
slicer.util.updateMarkupsControlPointsFromArray(muscle_cloud, muscle_df[['r','a','s']].to_numpy())
for i, name in enumerate(muscle_df.index):
    muscle_cloud.SetNthFiducialLabel(i, name)



<div class="alert alert-block alert-info">
<b>In Slicer GUI</b>
Clone muscles_orig, name the new object 'muscles_changed', colour purple
</div>

## Change muscles_orig in Slicer

<div class="alert alert-block alert-info">
<b>In Slicer GUI</b>

<p>adjust positions of muscles that are way off. Usually these are :</p> 
<li>sartor_1 and sartor_2</li>
<li>addmagisch_1</li>
<li>vas_med_r-P5 and vas_med_l-P5</li>
<li>bfsh_r-P2 and bfsh_l-P2</li>
<li>soleus_r-P1 and soleus_l-P1</li>
<li>tibanter_r-P3 and tibanter_l-P3</li>

</div>

In [18]:
bone_vals = np.array([pd.to_numeric(row) for row in bone_df[['r','a','s']].to_numpy()])
mscl_vals = np.array([pd.to_numeric(row) for row in muscle_df[['r','a','s']].to_numpy()])
wrp_vals = np.array([pd.to_numeric(row) for row in wraps_translation_df[['r','a','s']].to_numpy()])

In [19]:
muscles_changed = slicer.util.getNode('muscles_changed')
muscles_changed_list = slicer.util.arrayFromMarkupsControlPoints(muscles_changed)
muscles_changed_list.shape

(224, 3)

## Calculate changes and warp remaining muscles and wrapping surfaces

In [20]:
changes = []
for i, name in enumerate(muscle_df.index):
    a1 = np.around(muscles_changed_list[i], decimals=2)
    a2 = np.around(pd.to_numeric(muscle_df.loc[name, ['r','a','s']], downcast='float').to_numpy(), decimals=2)
    if np.count_nonzero(np.around(a1 - a2, decimals = 2))!=0.:
       changes.append(i)
       
res_list = [inds['muscle']['all'][i] for i in changes]
inds_for_tps = inds['bone']['all'] + res_list

changed = np.concatenate([bone_vals, muscles_changed_list[changes]])
orig = np.concatenate([bone_vals,mscl_vals[changes]])

In [21]:
len(changes)

62

In [22]:
tps_spline = ThinPlateSpline(alpha = 0.02)
tps_spline.fit(orig, changed)
# tps_spline.transform(self.osim_bone[['r','a','s']].to_numpy())
new_muscles = tps_spline.transform(mscl_vals)
new_wraps = tps_spline.transform(wrp_vals)

## Output results to Slicer

In [23]:
new_muscle_cloud = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode")
slicer.util.updateMarkupsControlPointsFromArray(new_muscle_cloud, new_muscles)
for i, name in enumerate(muscle_df.index):
    new_muscle_cloud.SetNthFiducialLabel(i, name)

## Record results to the control folder

In [24]:
new_muscles_df = muscle_df.copy()
new_muscles_df[['r','a','s']] = new_muscles
new_muscles_df.to_csv('new_muscles.csv')

In [25]:
new_wraps_df = wraps_translation_df.copy()
new_wraps_df[['r','a','s']] = new_wraps
new_wraps_df.to_csv('new_wraps.csv')


In [26]:
new_muscles_df = pd.read_csv('new_muscles.csv', index_col= 'name')
new_wraps_df = pd.read_csv('new_wraps.csv', index_col= 'name')